In [1]:
!pip install category-encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.6 MB/s eta 0:00:00


In [37]:
import pandas as pd
import requests as rq
import json as js
from time import sleep
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import category_encoders as ce
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
warnings.filterwarnings('ignore')
import joblib

# API Call

In [3]:
class BestTimeToFertilize:
    # Define the base URL for the weather API and the API key as class constants
    __BASE_URL = "https://api.weatherbit.io/v2.0/forecast/daily?"
    __API_KEY = "051d187cb45f4da7835e3fcd695dc707"

    def __init__(self, city_name = 'Nagpur', state_name = 'Maharashtra', days = 7):
        # Initialize the class with default values for location (city, state), country, and days of forecast
        self.city_name = '+'.join(city_name.lower().strip().split())  # Format and store the city name
        self.state_name = '+'.join(state_name.lower().strip().split())  # Format and store the state name
        self.country_name = 'IN'  # Country code (India)
        self.days = days  # Number of days to forecast
        self.response = None  # Initialize the API response variable
        self.response_code = None  # Initialize the API response code variable
        self.weather_data = list()  # List to store weather data

    def api_caller(self):
        try:
            # Build the complete API URL with parameters and make the API call
            complete_url = "{0}city={1}&state={2}&country={3}&key={4}&days={5}".format(
                self.__BASE_URL, self.city_name, self.state_name, self.country_name, self.__API_KEY, self.days
            )
            print(complete_url)  # Print the API URL for debugging purposes
            self.response = rq.get(complete_url)  # Make the API request
            sleep(5)  # Introduce a 5-second delay to avoid overloading the API
            self.response_code = self.response.status_code  # Get the API response code
            return self.response_code  # Return the API response code
        except Exception as msg:
            print("api_caller():", msg)
            return -1

    def is_api_call_success(self):
        # Check if the API call was successful based on the response code
        if self.response_code == 200:
            return True  # API call was successful
        elif self.response_code == 204:
            print('Oops! It seems there was an issue with the API call. Please check your input and try again later.')
        return False  # API call was not successful

    def json_file_bulider(self):
        try:
            # Parse the API response JSON and save it to a JSON file
            json_obj = self.response.json()
            with open('weather_data.json', 'w') as file:
                js.dump(json_obj, file, indent = 1, sort_keys = True)
            print("weather_data.json file built successfully")
        except Exception as msg:
            print("json_bulider():", msg)


    def best_time_fertilize(self):
        # Parse the JSON response from the API
        json_obj = self.response.json()

        print("City:", json_obj['city_name'], "\n")

        prolonged_precip = 0
        prolonged_prob = 0
        heavy_rain_2d = False
        heavy_rain_chance_2d = 0
        precip_2d = 0
        precip_chance_2d = 0

        # Loop through the forecast data for the specified number of days
        for i in range(self.days):
            date = json_obj['data'][i]['datetime']
            temp = json_obj['data'][i]['temp']
            rh = json_obj['data'][i]['rh']
            precip = json_obj['data'][i]['precip']
            prob = json_obj['data'][i]['pop']
            w_code = json_obj['data'][i]['weather']['code']
            w_desc = json_obj['data'][i]['weather']['description']
            i_code = json_obj['data'][i]['weather']['icon']
            prolonged_precip += precip
            prolonged_prob += prob

            count_2d = 0
            if i < 2:
                precip_2d += precip
                precip_chance_2d += prob

                # Check if the weather code indicates heavy rain
                if w_code in [202, 233, 502, 521, 522]:
                    heavy_rain_2d = True
                    heavy_rain_chance_2d += prob
                    count_2d += 1
                    heavy_rain_chance_2d //= count_2d

            # Create a dictionary to store weather data for the day
            di = {
                "Date": str(date),
                "Temperature": str(temp),
                "Relative Humidity": str(rh),
                "Rainfall": str(precip),
                "Probability of Precipitation": str(prob),
                "Weather code": str(w_code),
                "Weather Description": str(w_desc),
                "Icon code": str(i_code)
            }

            # Append the daily weather data to the list
            self.weather_data.append(di)

            print("Date:", date)
            print("Temperature:", temp)
            print("Relative Humidity:", rh)
            print("Rainfall:", precip)
            print("Probability of Precipitation:", prob)
            print("Weather code:", w_code, "->", w_desc)
            print()

        # Calculate the average probability of precipitation over the specified days
        prolonged_prob //= self.days
        precip_chance_2d //= 2

        print("-"*80)
        print("Rainfall Forecast for the Next 2 Days (including today):", precip_2d)
        print("Probability of Rain for the Next 2 Days (including today):", precip_chance_2d)
        print()

        # Check for heavy rain and issue a warning if heavy rain is expected
        if heavy_rain_2d:
            print("*"*21, "Warning !!!", "*"*21)
            print("Heavy Rain Chances:", heavy_rain_chance_2d)
            print("Heavy Rainfall puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

        print("Prolonged Precipitation:", prolonged_precip)
        print("Prolonged Precipitation Probability:", prolonged_prob)
        print()

        # Check if prolonged heavy rainfall is expected and issue a warning if so
        if prolonged_precip > 12.7 and prolonged_prob >= 50:
            print("*"*21, "Warning !!!", "*"*21)
            print("Prolonged Rainfall of greater than 12.7 mm puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

In [4]:
if __name__ == '__main__':
    # Create an instance of the BestTimeToFertilize class for Mumbai, Maharashtra
    obj = BestTimeToFertilize('Mumbai', 'Maharashtra')

    # Make the API call and print the response code
    print(obj.api_caller())

    # Check if the API call was successful
    if obj.is_api_call_success():
        # Retrieve and display the best time to fertilize information
        obj.best_time_fertilize()

        # Build a JSON file containing weather data
        obj.json_file_bulider()
        # print(obj.weather_data)


https://api.weatherbit.io/v2.0/forecast/daily?city=mumbai&state=maharashtra&country=IN&key=051d187cb45f4da7835e3fcd695dc707&days=7
200
City: Mumbai 

Date: 2023-09-18
Temperature: 26.7
Relative Humidity: 86
Rainfall: 37.375
Probability of Precipitation: 95
Weather code: 502 -> Heavy rain

Date: 2023-09-19
Temperature: 28.4
Relative Humidity: 83
Rainfall: 27.75
Probability of Precipitation: 90
Weather code: 202 -> Thunderstorm with heavy rain

Date: 2023-09-20
Temperature: 28.6
Relative Humidity: 83
Rainfall: 19.6875
Probability of Precipitation: 85
Weather code: 202 -> Thunderstorm with heavy rain

Date: 2023-09-21
Temperature: 28.4
Relative Humidity: 84
Rainfall: 8.75
Probability of Precipitation: 75
Weather code: 201 -> Thunderstorm with rain

Date: 2023-09-22
Temperature: 28.1
Relative Humidity: 86
Rainfall: 10.125
Probability of Precipitation: 75
Weather code: 501 -> Moderate rain

Date: 2023-09-23
Temperature: 27.9
Relative Humidity: 87
Rainfall: 10
Probability of Precipitation: 7

# Data

In [5]:
# data = './app/data/Nutrient_recommendation.csv'
data = '/content/Nutrient_recommendation.csv'
df = pd.read_csv(data, header=None)
df.shape

(2201, 7)

In [6]:
df.head(10)

,0,1,2,3,4,5,6
0,Crop,Temperature,Humidity,Rainfall,Label_N,Label_P,Label_K
1,rice,20.87974371,82.00274423,202.9355362,90,42,43
2,rice,21.77046169,80.31964408,226.6555374,85,58,41
3,rice,23.00445915,82.3207629,263.9642476,60,55,44
4,rice,26.49109635,80.15836264,242.8640342,74,35,40
5,rice,20.13017482,81.60487287,262.7173405,78,42,42
6,rice,23.05804872,83.37011772,251.0549998,69,37,42
7,rice,22.70883798,82.63941394,271.3248604,69,55,38
8,rice,20.27774362,82.89408619,241.9741949,94,53,40
9,rice,24.51588066,83.5352163,230.4462359,89,54,38


In [7]:
df.columns = ['Crop', 'Temperature', 'Humidity', 'Rainfall', 'Label_N', 'Label_P', 'Label_K']
df.drop(df.index[:1], inplace=True)
df.head(10)

,Crop,Temperature,Humidity,Rainfall,Label_N,Label_P,Label_K
1,rice,20.87974371,82.00274423,202.9355362,90,42,43
2,rice,21.77046169,80.31964408,226.6555374,85,58,41
3,rice,23.00445915,82.3207629,263.9642476,60,55,44
4,rice,26.49109635,80.15836264,242.8640342,74,35,40
5,rice,20.13017482,81.60487287,262.7173405,78,42,42
6,rice,23.05804872,83.37011772,251.0549998,69,37,42
7,rice,22.70883798,82.63941394,271.3248604,69,55,38
8,rice,20.27774362,82.89408619,241.9741949,94,53,40
9,rice,24.51588066,83.5352163,230.4462359,89,54,38
10,rice,23.22397386,83.03322691,221.2091958,68,58,38


In [8]:
df[['Temperature', 'Humidity', 'Rainfall']] = df[['Temperature', 'Humidity', 'Rainfall']].astype(float)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 1 to 2200
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Crop         2200 non-null   object 
 1   Temperature  2200 non-null   float64
 2   Humidity     2200 non-null   float64
 3   Rainfall     2200 non-null   float64
 4   Label_N      2200 non-null   object 
 5   Label_P      2200 non-null   object 
 6   Label_K      2200 non-null   object 
dtypes: float64(3), object(4)
memory usage: 120.4+ KB


In [10]:
df.isnull().sum()

Crop           0
Temperature    0
Humidity       0
Rainfall       0
Label_N        0
Label_P        0
Label_K        0
dtype: int64

In [11]:
# Separate features (X) and labels (y) for three nutrients: Nitrogen (N), Phosphorus (P), and Potassium (K).

# Drop the columns 'Label_N', 'Label_P', and 'Label_K' to create the feature set X.
X = df.drop(['Label_N', 'Label_P', 'Label_K'], axis=1)

# Create separate label sets (y1, y2, y3) for Nitrogen (N), Phosphorus (P), and Potassium (K).
y1 = df['Label_N']
y2 = df['Label_P']
y3 = df['Label_K']

# Concatenate the label sets into a single 2D array 'y' with the labels for each nutrient in separate columns.
y = np.concatenate([[y1], [y2], [y3]], axis=0).T
y


array([['90', '42', '43'],
       ['85', '58', '41'],
       ['60', '55', '44'],
       ...,
       ['118', '33', '30'],
       ['117', '32', '34'],
       ['104', '18', '30']], dtype=object)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape

((1760, 4), (440, 4))

In [13]:
y_train.shape, y_test.shape

((1760, 3), (440, 3))

In [14]:
# Separate the training labels into three arrays for each nutrient: Nitrogen (N), Phosphorus (P), and Potassium (K).

# Training labels for Nitrogen (N)
y1_train = y_train[:, 0]

# Training labels for Phosphorus (P)
y2_train = y_train[:, 1]

# Training labels for Potassium (K)
y3_train = y_train[:, 2]

# Separate the test labels into three arrays for each nutrient: Nitrogen (N), Phosphorus (P), and Potassium (K).

# Test labels for Nitrogen (N)
y1_test = y_test[:, 0]

# Test labels for Phosphorus (P)
y2_test = y_test[:, 1]

# Test labels for Potassium (K)
y3_test = y_test[:, 2]

In [15]:
X_train.head(10)

,Crop,Temperature,Humidity,Rainfall
1528,apple,22.812125,91.518617,107.855225
698,mungbean,28.174587,83.696593,37.246465
1233,grapes,35.893075,82.668507,66.539466
641,mungbean,27.336844,88.502291,51.098026
1830,coconut,27.148653,96.663552,149.243350
1988,cotton,24.688548,78.512070,69.311536
1698,orange,27.681673,94.473169,113.999515
1142,mango,30.914715,49.929639,90.140478
24,rice,21.052536,82.678395,233.107582
2139,coffee,23.426116,64.106515,122.684741


In [16]:
# Create an empty dictionary to store the mapping of crop names to unique keys.
mapping = dict()

# Store the crop names and their corresponding keys in a csv file.
with open("/content/mapped_crops.csv", "w") as f:
    f.write("Crops,Key\n")

    # Enumerate through unique crop names in the DataFrame and assign a unique key to each crop.
    for i, crop in enumerate(np.unique(df[['Crop']]), 1):
        # Store the mapping of crop name to key in the 'mapping' dictionary.
        mapping[crop] = i

        # Write each crop name and its corresponding key to the CSV file.
        f.write("%s,%d\n" % (crop, i))

    # Add a mapping for 'NA' (missing data) to NaN and write it to the CSV file.
    mapping['NA'] = np.nan
    f.write("NA,nan")

print(mapping)


{'apple': 1, 'banana': 2, 'blackgram': 3, 'chickpea': 4, 'coconut': 5, 'coffee': 6, 'cotton': 7, 'grapes': 8, 'jute': 9, 'kidneybeans': 10, 'lentil': 11, 'maize': 12, 'mango': 13, 'mothbeans': 14, 'mungbean': 15, 'muskmelon': 16, 'orange': 17, 'papaya': 18, 'pigeonpeas': 19, 'pomegranate': 20, 'rice': 21, 'watermelon': 22, 'NA': nan}


In [17]:
# Create a list of dictionaries specifying the column to encode ('Crop') and its corresponding mapping.
ordinal_cols_mapping = [{"col": "Crop", "mapping": mapping}, ]

# Create an OrdinalEncoder instance with the specified mapping.
encoder = ce.OrdinalEncoder(cols='Crop', mapping=ordinal_cols_mapping, return_df=True)

# Fit and transform the training data using the encoder.
X_train = encoder.fit_transform(X_train)

# Transform the test data using the same encoder.
X_test = encoder.transform(X_test)

# Display the first few rows of the transformed training data.
X_train.head()


,Crop,Temperature,Humidity,Rainfall
1528,1,22.812125,91.518617,107.855225
698,15,28.174587,83.696593,37.246465
1233,8,35.893075,82.668507,66.539466
641,15,27.336844,88.502291,51.098026
1830,5,27.148653,96.663552,149.243350


# Training

### Random Forest

In [18]:
# Import the RandomForestRegressor from sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

# Create a RandomForestRegressor instance with a random seed for reproducibility
regressor = RandomForestRegressor(random_state=42)

# Create a GridSearchCV instance for hyperparameter tuning
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the grid search to the training data for y1 (Label_N)
grid_search.fit(X_train, y1_train)

# Get the best parameters and best regressor from the grid search
best_params_rf = grid_search.best_params_
best_regressor_rf = grid_search.best_estimator_

# print("Best Regressor : ",best_regressor_rf)
# print("Best Parameters : ",best_params_rf)


In [19]:
print(" Using Random Forest Regressor \n")
# For Nitrogen

# Create a RandomForestRegressor for Nitrogen using the best parameters
rf1 = RandomForestRegressor(**best_params_rf)
rf1.fit(X_train, y1_train)
y1_pred = rf1.predict(X_test)

print(" For Nitrogen ")

# Calculate and print Mean Absolute Error for Nitrogen
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))

# Calculate and print Mean Squared Error for Nitrogen
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))

# Calculate and print Root Mean Squared Error for Nitrogen
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

# Calculate and print RMSE (Root Mean Squared Error) for Nitrogen
rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Phosphorus

# Create a RandomForestRegressor for Phosphorus using the best parameters
rf2 = RandomForestRegressor(**best_params_rf)
rf2.fit(X_train, y2_train)
y2_pred = rf2.predict(X_test)

print(" For Phosphorus ")

# Calculate and print Mean Absolute Error for Phosphorus
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))

# Calculate and print Mean Squared Error for Phosphorus
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))

# Calculate and print Root Mean Squared Error for Phosphorus
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

# Calculate and print RMSE (Root Mean Squared Error) for Phosphorus
rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

# Create a RandomForestRegressor for Potassium using the best parameters
rf3 = RandomForestRegressor(**best_params_rf)
rf3.fit(X_train, y3_train)
y3_pred = rf3.predict(X_test)

print(" For Potassium ")

# Calculate and print Mean Absolute Error for Potassium
print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))

# Calculate and print Mean Squared Error for Potassium
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))

# Calculate and print Root Mean Squared Error for Potassium
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

# Calculate and print RMSE (Root Mean Squared Error) for Potassium
rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()


 Using Random Forest Regressor 

 For Nitrogen 
Mean Absolute Error: 10.845925335064223
Mean Squared Error: 160.06090342025985
Root Mean Squared Error: 12.651517830689718
RMSE:  12.652

 For Phosphorus 
Mean Absolute Error: 7.017587165537995
Mean Squared Error: 72.05081044807227
Root Mean Squared Error: 8.488274880567445
RMSE:  8.488

 For Potassium 
Mean Absolute Error: 2.6638736040035385
Mean Squared Error: 10.223668549963838
Root Mean Squared Error: 3.1974471926779087
RMSE:  3.197



### Gradient Boosting

In [20]:
from sklearn.ensemble import GradientBoostingRegressor as xgboost
from xgboost import XGBRegressor

# Define the hyperparameter grid for GridSearch
param_grid = {
    'n_estimators': [50, 75, 100, 125],
    'max_depth': [5, 10, 15, 20],
}

# Create a GradientBoostingRegressor
gbr = XGBRegressor(random_state=42)

# Perform GridSearchCV to find the best parameters for Nitrogen prediction
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y1_train)

# Get the best parameters and best regressor for Nitrogen prediction
best_params_gb = grid_search.best_params_
best_regressor_gb = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_gb)
# print("Best Parameter : ", best_params_gb)


In [21]:
print(" Using Gradient Boosting Regressor \n")

# For Nitrogen
gb1 = XGBRegressor(**best_params_gb)
gb1.fit(X_train, y1_train)
y1_pred = gb1.predict(X_test)

print(" For Nitrogen ")

# Print evaluation metrics for Nitrogen prediction
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus
gb2 = XGBRegressor(**best_params_gb)
gb2.fit(X_train, y2_train)
y2_pred = gb2.predict(X_test)

print(" For Phosphorus ")

# Print evaluation metrics for Phosphorus prediction
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium
gb3 = XGBRegressor(**best_params_gb)
gb3.fit(X_train, y2_train)
y2_pred = gb3.predict(X_test)

print(" For Potassium")

# Print evaluation metrics for Potassium prediction
print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()


 Using Gradient Boosting Regressor 

 For Nitrogen 
Mean Absolute Error: 11.303323269974102
Mean Squared Error: 177.32510311609462
Root Mean Squared Error: 13.31634721371047
RMSE :  13.316

 For Phosphorus 
Mean Absolute Error: 7.246350728381763
Mean Squared Error: 77.37517346489697
Root Mean Squared Error: 8.796315902973072
RMSE:  8.796

 For Potassium
Mean Absolute Error: 2.6638736040035385
Mean Squared Error: 10.223668549963838
Root Mean Squared Error: 3.1974471926779087
RMSE:  3.197



## Light GBM

In [22]:
# Import the LGBMRegressor from the lightgbm library

from lightgbm import LGBMRegressor

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 75, 100, 125],
    'max_depth': [5, 10, 15, 20],
    'min_child_samples': [2, 4, 6, 8],
    'min_child_weight': [1e-4, 1e-3, 1e-2, 1e-1],
}

# Create a LGBMRegressor with a random seed
regressor = LGBMRegressor(random_state=42)

# Perform a GridSearchCV for hyperparameter tuning with 5-fold cross-validation and neg_mean_squared_error scoring
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y1_train)

# Get the best parameters and best estimator from the grid search
best_params_lgb = grid_search.best_params_
best_regressor_lgb = grid_search.best_estimator_

# Print out the best parameters and best estimator, if needed
# print("Best Parameters : ", best_regressor_lgb)
# print("Best Parameters : ", best_params_lgb)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 788
[LightGBM] [Info] Number of data points in the train set: 1760, number of used features: 4
[LightGBM] [Info] Start training from score 50.420455


In [23]:
# Print a message indicating the use of Light GBM Regressor
print(" Using Light GBM Regressor \n")

# For Nitrogen

# Create a LGBMRegressor object with the best parameters and fit it to the training data
lgb1 = LGBMRegressor(**best_params_lgb)
lgb1.fit(X_train, y1_train)
# Predict the target variable for the test data
y1_pred = lgb1.predict(X_test)

# Print information for Nitrogen predictions
print(" For Nitrogen ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

# Create a LGBMRegressor object with the best parameters and fit it to the training data
lgb2 = LGBMRegressor(**best_params_lgb)
lgb2.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = lgb2.predict(X_test)

# Print information for Phosphorus predictions
print(" For Phosphorus ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

# Create a LGBMRegressor object with the best parameters and fit it to the training data
lgb3 = LGBMRegressor(**best_params_lgb)
lgb3.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = lgb3.predict(X_test)

# Print information for Potassium predictions
print(" For Potassium")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()


 Using Light GBM Regressor 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 788
[LightGBM] [Info] Number of data points in the train set: 1760, number of used features: 4
[LightGBM] [Info] Start training from score 50.420455
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
 For Nitrogen 
Mean Absolute Error: 11.1142115684178
Mean Squared Error: 176.0039429072279
Root Mean Squared Error: 13.266647764496799
RMSE :  13.267

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 

## Robust Regression (RANSAC)


In [24]:
from sklearn.linear_model import LinearRegression, RANSACRegressor

# Define the hyperparameter grid for GridSearch
param_grid = {
    'min_samples' : [40, 50, 60, 70, 80],
    'max_trials' : [50, 75, 100, 125, 150],
    'residual_threshold' : [5, 10, 15, 20, 25]
}

# Create a RANSACRegressor with a LinearRegression base estimator
rrr = RANSACRegressor(random_state=42,
                      base_estimator=LinearRegression())

# Perform a GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rrr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y1_train)

# Get the best hyperparameters and estimator
best_params_rrr = grid_search.best_params_
best_regressor_rrr = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_rrr)
# print("Best parameters : ", best_params_rrr)


In [25]:
# Print a message indicating the use of Robust Regressor
print(" Using Robust Regressor \n")

# For Nitrogen

# Create a RANSACRegressor object with the best parameters and fit it to the training data
rr1 = RANSACRegressor(**best_params_rrr)
rr1.fit(X_train, y1_train)
# Predict the target variable for the test data
y1_pred = rr1.predict(X_test)

# Print information for Nitrogen predictions
print(" For Nitrogen ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

# Create a RANSACRegressor object with the best parameters and fit it to the training data
rr2 = RANSACRegressor(**best_params_rrr)
rr2.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = rr2.predict(X_test)

# Print information for Phosphorus predictions
print(" For Phosphorus ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

# Create a RANSACRegressor object with the best parameters and fit it to the training data
rr3 = RANSACRegressor(**best_params_rrr)
rr3.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = rr3.predict(X_test)

# Print information for Potassium predictions
print(" For Potassium")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

# Format and print the RMSE
rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()


 Using Robust Regressor 

 For Nitrogen 
Mean Absolute Error: 30.120814710147915
Mean Squared Error: 1437.8707118966327
Root Mean Squared Error: 37.919265708827126
RMSE :  37.919

 For Phosphorus 
Mean Absolute Error: 21.49840796674776
Mean Squared Error: 1029.7252721901145
Root Mean Squared Error: 32.08933268533509
RMSE:  32.089

 For Potassium
Mean Absolute Error: 2.6638736040035385
Mean Squared Error: 10.223668549963838
Root Mean Squared Error: 3.1974471926779087
RMSE:  3.197



### ElasticNet

In [26]:
# Import the ElasticNet class from sklearn.linear_model

from sklearn.linear_model import ElasticNet

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.35, 0.65, 0.95],
}

# Create an ElasticNet regressor with a random seed
regressor = ElasticNet(random_state=42)

# Perform a GridSearchCV for hyperparameter tuning with 5-fold cross-validation and neg_mean_squared_error scoring
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator from the grid search
best_params_en = grid_search.best_params_
best_regressor_en = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_en)
# print("Best Parameters : ", best_params_en)


In [27]:
# Print a message indicating the use of Elastic Net Regressor
print(" Using Elastic Net \n")

# For Nitrogen

# Create an ElasticNet object with the best parameters and fit it to the training data
en1 = ElasticNet(**best_params_en)
en1.fit(X_train, y1_train)
# Predict the target variable for the test data
y1_pred = en1.predict(X_test)

# Print information for Nitrogen predictions
print(" For Nitrogen ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

# Create an ElasticNet object with the best parameters and fit it to the training data
en2 = ElasticNet(**best_params_en)
en2.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = en2.predict(X_test)

# Print information for Phosphorus predictions
print(" For Phosphorus ")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

# Create an ElasticNet object with the best parameters and fit it to the training data
en3 = ElasticNet(**best_params_en)
en3.fit(X_train, y2_train)
# Predict the target variable for the test data
y2_pred = en3.predict(X_test)

# Print information for Potassium predictions
print(" For Potassium")

# Calculate and print the Mean Absolute Error
print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
# Calculate and print the Mean Squared Error
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
# Calculate and print the Root Mean Squared Error
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

# Format and print the RMSE with three decimal places
rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()


 Using Elastic Net 

 For Nitrogen 
Mean Absolute Error: 31.60354143291646
Mean Squared Error: 1320.2631097812791
Root Mean Squared Error: 36.3354249979449
RMSE :  36.335

 For Phosphorus 
Mean Absolute Error: 21.040292495904744
Mean Squared Error: 793.1873074777342
Root Mean Squared Error: 28.163581226075177
RMSE:  28.164

 For Potassium
Mean Absolute Error: 2.6638736040035385
Mean Squared Error: 10.223668549963838
Root Mean Squared Error: 3.1974471926779087
RMSE:  3.197



### Best Model

In [28]:
 # Save the Nitrogen Random Forest Regressor model to disk with the filename './rf_N-v1.pkl'
joblib.dump(rf1, './rf_N-v1.pkl')

# Save the Phosphorus LightGBM Regressor model to disk with the filename './lgbm_P-v1.pkl'
joblib.dump(lgb2, './lgbm_P-v1.pkl')

# Save the Potassium Random Forest Regressor model to disk with the filename './rf_K-v1.pkl'
joblib.dump(rf3, './rf_K-v1.pkl')


['./rf_K-v1.pkl']